In [1]:
#这个脚本希望将三亚历年的风向数据转换为风向风速数据
import numpy as np
import pandas as pd
import json

In [19]:
#提取三亚
fileName = 'city_features.csv'
totalData = pd.read_csv(fileName)
syData = totalData[0:1]
syData = syData.drop(labels='Unnamed: 0', axis = 1)

In [20]:
#写入csv中，列为污染物名，行为天数date
windType = ['U','V']
syWindData = pd.DataFrame(columns=['U','V'])
for i in range(syData.shape[1]):
    #i表示天数
    tmp_list = json.loads(syData[syData.columns[i]][0])
    tmp_dict = {}
    for j in range(len(windType)):
        #第j种污染物
        tmp_dict[windType[j]] = tmp_list[j+6]
    syWindData = syWindData.append(tmp_dict, ignore_index=True)

In [35]:
#编写一个函数根据其绝对值确定风力级别
#为方便，没有严格按照标准细分
#0~2为0级，3~4为1级，5~6为2级，7及以上为3级
#依数据分布情况，可能有所改变
def getWindDegree(windSpeed):
    windSpeed = abs(windSpeed)
    if(windSpeed < 3.4):
        return 0
    if(windSpeed < 8.0):
        return 1
    return 2
    
#从U，V得到这个风是哪个方向，u是纬向，v是经向
#先返回纬向，再返回经向
def getWindDirec(u, v):
    if u>=0 and v>=0:
        return ['E', 'N', 'NE']
    if u>=0 and v<0:
        return ['E', 'S', 'SE']
    if u<0 and v>=0:
        return ['W', 'N', 'NW']
    if u<0 and v<0:
        return ['W', 'S', 'SW']  

In [36]:
#数据是得到了，但是怎样确定风向风速的计算方式？
#我觉得粗略一点也没有关系
#不管了，默认正值表示西风，即从西向东，或者南风，即从南向北的风力大小
syWindFre = pd.DataFrame(columns=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
tmpDict = {
    'N': 0,
    'NE': 0,
    'E': 0,
    'SE': 0,
    'S': 0,
    'SW': 0,
    'W': 0,
    'NW': 0
}
for i in range(3):
    syWindFre = syWindFre.append(tmpDict,ignore_index=True)

In [37]:
#接下来进行计数频率的统计，U纬向，V经向
for i in range(len(syWindData)):
    u = syWindData['U'][i]
    v = syWindData['V'][i]
    direcs = getWindDirec(u,v)
    #开始计数
    u = abs(u)
    v = abs(v)
    middleSpeed = (u+v)*pow(0.5, 0.5)
    speeds = [u,v,middleSpeed]
    for j in range(3):
        tmpDirec = direcs[j]
        tmpSpeed = speeds[j]
        tmpDegree = getWindDegree(tmpSpeed)
        syWindFre[tmpDirec][tmpDegree] = syWindFre[tmpDirec][tmpDegree] + 1

In [39]:
syWindFre.to_csv('syWindPre.csv', index=False)

In [32]:
744	151	282	47	638	187	702	308

0.7071067811865476